In [ ]:
# --- Cell 1: Cài đặt thư viện ---
!pip install -U transformers datasets accelerate bitsandbytes flash-attn scikit-learn peft trl protobuf huggingface_hub

In [ ]:
!pip install sentencepiece

In [ ]:
from huggingface_hub import login
#login('')

In [ ]:
# --- Cell 2: Import thư viện ---
import os
import json
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch
import transformers
print(f"Transformers version: {transformers.__version__}")
print(f"CUDA is available: {torch.cuda.is_available()}")
print(f"CUDA device count: {torch.cuda.device_count()}")
print(f"CUDA device name: {torch.cuda.get_device_name(0)}")

# --- Cell 3: Đăng nhập vào Hugging Face Hub ---
from huggingface_hub import login

# --- Cell 4: Chuyển dữ liệu txt sang dataset ---
DATA_DIR = 'dermnetnz_qa'
data = []
for cat in os.listdir(DATA_DIR):
    cat_dir = os.path.join(DATA_DIR, cat)
    for fname in os.listdir(cat_dir):
        with open(os.path.join(cat_dir, fname), encoding='utf-8') as f:
            qa = {}
            for line in f:
                if line.startswith('Q:'):
                    qa['question'] = line[2:].strip()
                elif line.startswith('A:'):
                    qa['answer'] = line[2:].strip()
                elif line.strip() == '' and qa:
                    if 'question' in qa and 'answer' in qa:
                        data.append(qa)
                    qa = {}
            if 'question' in qa and 'answer' in qa:
                data.append(qa)

print(f"Tổng số cặp QA: {len(data)}")

# --- Cell 5: Chia tập train/val/test ---
test_size = 20
val_ratio = 0.1

trainval_data, test_data = train_test_split(data, test_size=test_size, random_state=42)
val_size = int(len(trainval_data) * val_ratio)
train_data, val_data = train_test_split(trainval_data, test_size=val_size, random_state=42)

print(f"Số cặp train: {len(train_data)}, Số cặp val: {len(val_data)}, Số cặp test: {len(test_data)}")

# --- Cell 6: Tạo dataset ---
from datasets import Dataset

train_dataset = Dataset.from_list([{'question': item['question'], 'answer': item['answer']} for item in train_data])
val_dataset = Dataset.from_list([{'question': item['question'], 'answer': item['answer']} for item in val_data])
test_dataset = Dataset.from_list([{'question': item['question'], 'answer': item['answer']} for item in test_data])

# --- Cell 7: Tải LLaMA-2 7B và lượng tử hóa ---
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

# Cấu hình lượng tử hóa 4-bit để tiết kiệm VRAM
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Tải tokenizer - cần sử dụng trust_remote_code
tokenizer = AutoTokenizer.from_pretrained(
    model_id, 
    use_fast=False,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Tải mô hình với cấu hình lượng tử hóa
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,  # Sử dụng lượng tử hóa 4-bit
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="flash_attention_2"  # Thay thế use_flash_attention_2
)

# --- Cell 8: Thêm LoRA adapter ---
from peft import LoraConfig, get_peft_model

# Cấu hình LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    # Target modules dành riêng cho Mixtral
    target_modules=[
        "q_proj", 
        "k_proj", 
        "v_proj", 
        "o_proj",
        "gate_proj",  # Thêm layers đặc trưng cho MoE
        "up_proj", 
        "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Chuẩn bị mô hình và thêm LoRA adapter
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# --- Cell 9: Chuẩn bị dữ liệu huấn luyện ---
from transformers import DataCollatorForSeq2Seq

# Format prompt theo chuẩn Mixtral
def generate_prompt(example):
    return f"<s>[INST] {example['question']} [/INST] {example['answer']}</s>"
    # Hoặc format ChatML nếu cần
    # return f"<s><im_start>user\n{example['question']}<im_end>\n<im_start>assistant\n{example['answer']}<im_end></s>"

# Tokenize datasets
def tokenize_function(example):
    prompt = generate_prompt(example)
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,  # Giảm kích thước để tiết kiệm VRAM
        padding=False,
        return_tensors=None,
    )
    
    # Đặt labels bằng input_ids
    result["labels"] = result["input_ids"].copy()
    return result

# Áp dụng tokenize
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=False)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=False)

# Data collator với padding
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    padding=True,
    return_tensors="pt",
)

# --- Cell 10: Huấn luyện mô hình với LoRA ---
from transformers import Trainer, TrainingArguments

# Tính số bước cho mỗi epoch
steps_per_epoch = len(tokenized_train_dataset) // (8 * 4)  # batch_size * gradient_accumulation_steps
if steps_per_epoch == 0:
    steps_per_epoch = 1




In [ ]:
# Cấu hình huấn luyện cho Mixtral (giảm batch size do mô hình lớn hơn)
training_arguments = TrainingArguments(
    output_dir="./mixtral-8x7b-lora-dermnetz",
    run_name="mixtral-8x7b-lora-finetune",
    per_device_train_batch_size=4,        # Giảm xuống từ 16
    per_device_eval_batch_size=4,         # Giảm xuống từ 16
    gradient_accumulation_steps=8,        # Tăng lên để bù đắp batch size nhỏ
    logging_steps=10,
    learning_rate=2e-4,
    num_train_epochs=2,
    eval_steps=steps_per_epoch,
    save_steps=400,
    bf16=True,
    bf16_full_eval=True,
    warmup_steps=steps_per_epoch//3,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="none",
    # Thêm gradient_checkpointing nếu cần
    gradient_checkpointing=True,
)

# Khởi tạo Trainer
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=data_collator,
    # Không sử dụng tokenizer parameter
)

# --- Cell 11: Huấn luyện ---
trainer.train()

In [ ]:
# --- Cell 12: Lưu model ---
# Chỉ lưu adapter LoRA - không lưu mô hình đầy đủ
model.save_pretrained("./mixtral-8x7b-lora-dermnetz", safe_serialization=True)
tokenizer.save_pretrained("./mixtral-8x7b-lora-dermnetz")


In [ ]:
# In kích thước của adapter LoRA đã lưu
!du -sh ./mixtral-8x7b-lora-dermnetz

In [ ]:
# --- Cell 12: Đánh giá loss trên validation set ---
import time
import torch

print("=== Đánh giá loss trên toàn bộ validation set ===")
print(f"Số lượng mẫu trong validation set: {len(val_dataset)}")

start_time = time.time()
eval_results = trainer.evaluate()
end_time = time.time()

print(f"Validation Loss: {eval_results['eval_loss']:.4f}")
print(f"Validation Perplexity: {torch.exp(torch.tensor(eval_results['eval_loss'])):.4f}")
print(f"Thời gian đánh giá: {end_time - start_time:.2f} giây")

In [ ]:
!pip install rouge_score

In [ ]:
# --- Cell: Test model với so sánh câu trả lời ---
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from rouge_score import rouge_scorer
import numpy as np
from tqdm import tqdm

# 1. Load base model và tokenizer (giống như cell inference)
model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(
    model_id, 
    use_fast=False,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 2. Load base model và adapter
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="flash_attention_2"
)
adapter_path = "./mixtral-8x7b-lora-dermnetz"  # Đường dẫn đến checkpoint
model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval()

# 3. Load ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# 4. Hàm tạo prompt
def generate_prompt(question):
    return f"<s>[INST] {question} [/INST]"

# 5. Hàm sinh câu trả lời
def generate_answer(question, max_new_tokens=512, temperature=0.7):
    prompt = generate_prompt(question)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.9,
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Xử lý đầu ra dựa trên format Mixtral
    response = response.split("[/INST]")[-1].strip()
    return response

# 6. Test với tập test data và tính điểm
results = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

# Lấy 10 mẫu từ test_dataset để kiểm tra (hoặc toàn bộ nếu ít hơn 10)
num_test_samples = min(len(test_dataset), 20)
for i in tqdm(range(num_test_samples)):
    sample = test_dataset[i]
    question = sample['question']
    ground_truth = sample['answer']
    
    # Sinh câu trả lời từ model
    model_answer = generate_answer(question)
    
    # Tính điểm ROUGE
    scores = scorer.score(ground_truth, model_answer)
    
    # Lưu kết quả
    results.append({
        'question': question,
        'ground_truth': ground_truth,
        'model_answer': model_answer,
        'rouge1': scores['rouge1'].fmeasure,
        'rouge2': scores['rouge2'].fmeasure,
        'rougeL': scores['rougeL'].fmeasure
    })
    
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

# 7. In kết quả chi tiết và điểm trung bình
print(f"===== TEST RESULTS (AVERAGE SCORES) =====")
print(f"ROUGE-1: {np.mean(rouge1_scores):.4f}")
print(f"ROUGE-2: {np.mean(rouge2_scores):.4f}")  
print(f"ROUGE-L: {np.mean(rougeL_scores):.4f}")
print("\n===== DETAILED RESULTS =====")

for i, result in enumerate(results):
    print(f"\n----- Example {i+1} -----")
    print(f"Q: {result['question']}")
    print(f"Ground truth: {result['ground_truth']}")
    print(f"Model answer: {result['model_answer']}")
    print(f"ROUGE-1: {result['rouge1']:.4f}")
    print(f"ROUGE-2: {result['rouge2']:.4f}")
    print(f"ROUGE-L: {result['rougeL']:.4f}")